## Исследование данных

In [1]:
import json
import pandas as pd

#### Импортирование данных Мурманска

In [2]:
a = []
for n in range(1, 50):
    content = pd.read_json(f"http://31.172.135.12/api/events?page={n}", lines=True)
    a.extend(content['data'][0])

In [3]:
df = pd.DataFrame(a)

In [4]:
df.to_csv('data.csv', index=False)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      735 non-null    int64  
 1   user_id                 0 non-null      object 
 2   title                   735 non-null    object 
 3   description             735 non-null    object 
 4   images                  0 non-null      object 
 5   started_at              734 non-null    object 
 6   stopped_at              735 non-null    object 
 7   created_at              735 non-null    object 
 8   updated_at              735 non-null    object 
 9   deleted_at              0 non-null      object 
 10  annotation              722 non-null    object 
 11  city_id                 735 non-null    int64  
 12  place_id                109 non-null    float64
 13  label                   712 non-null    object 
 14  status                  735 non-null    ob

In [6]:
nan_cols = df.columns[df.isna().all(0)]
nan_cols

Index(['user_id', 'images', 'deleted_at', 'age_restriction_id', 'season_id',
       'rating', 'user_rating', 'rating_count'],
      dtype='object')

In [7]:
df.drop(nan_cols, axis=1, inplace=True)

In [8]:
df.describe()

,id,city_id,place_id,category_id,tourism_type_id
count,735.000000,735.000000,109.000000,735.000000,36.000000
mean,429.512925,11.697959,184.963303,5.029932,3.805556
std,223.086729,1.907588,215.997168,2.313137,0.576663
min,23.000000,2.000000,39.000000,1.000000,2.000000
25%,242.500000,12.000000,86.000000,3.000000,4.000000
50%,437.000000,12.000000,109.000000,4.000000,4.000000
75%,621.500000,12.000000,117.000000,8.000000,4.000000
max,829.000000,17.000000,752.000000,10.000000,5.000000


In [9]:
df.nunique()

id                        735
title                     714
description               734
started_at                466
stopped_at                483
created_at                701
updated_at                735
annotation                680
city_id                    12
place_id                   22
label                      28
status                      1
category_id                 9
tourism_type_id             4
accessible_environment      2
adapter                     1
external_id               721
external_data             721
dtype: int64

In [10]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [11]:
import numpy as np
def word_averaging(wv, words):
    mean = np.zeros((wv.vector_size,))
    
    for word in words:
        if word in wv.vocab:
            mean += wv.get_vector(word)

    if all(mean == 0.):
        logging.warning("cannot compute similarity with no input %s", words)
        return mean

    mean = gensim.matutils.unitvec(mean)
    return mean

def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list])

#### Данные с kudago по Москве

In [12]:
import json
resp1 = json.load(open('response_1616889786114.json', encoding='utf-8'))

In [13]:
df = pd.DataFrame(([list(*p.items()) for p in resp1]), columns=['Description', 'spam'])

In [14]:
df.drop_duplicates(inplace=True)

In [15]:
!pip install html2text

In [16]:
import html2text

#### Предобработка

In [17]:
df.Description = df.Description.apply(html2text.html2text)
df.Description = df.Description.str.lower()
df.Description = df.Description.str.replace('[\W_]', ' ', regex=True)
df.Description = df.Description.apply(lambda x: [i for i in x.split() if len(i) > 1])

In [18]:
# conda install -c anaconda gensim

#### Обучение Word2Vec и усреднение векторов слов

In [22]:
from gensim.models import Word2Vec
import gensim

In [20]:
model = Word2Vec(df.Description, size=160, window=30, min_count=2, workers=12, iter=10, sg=1)

2021-03-28 11:13:35,788 : INFO : collecting all words and their counts
2021-03-28 11:13:35,788 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-28 11:13:35,891 : INFO : collected 72056 word types from a corpus of 478538 raw words and 3395 sentences
2021-03-28 11:13:35,899 : INFO : Loading a fresh vocabulary
2021-03-28 11:13:35,995 : INFO : effective_min_count=2 retains 33024 unique words (45% of original 72056, drops 39032)
2021-03-28 11:13:35,995 : INFO : effective_min_count=2 leaves 439506 word corpus (91% of original 478538, drops 39032)
2021-03-28 11:13:36,084 : INFO : deleting the raw counts dictionary of 72056 items
2021-03-28 11:13:36,084 : INFO : sample=0.001 downsamples 21 most-common words
2021-03-28 11:13:36,084 : INFO : downsampling leaves estimated 416385 word corpus (94.7% of prior 439506)
2021-03-28 11:13:36,156 : INFO : estimated required memory for 33024 words and 160 dimensions: 58782720 bytes
2021-03-28 11:13:36,156 : INFO : resettin

2021-03-28 11:13:57,594 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-28 11:13:57,594 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-28 11:13:57,626 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-28 11:13:57,626 : INFO : EPOCH - 5 : training on 478538 raw words (416293 effective words) took 3.3s, 124785 effective words/s
2021-03-28 11:13:58,857 : INFO : EPOCH 6 - PROGRESS: at 24.09% examples, 90257 words/s, in_qsize 24, out_qsize 0
2021-03-28 11:13:59,959 : INFO : EPOCH 6 - PROGRESS: at 63.83% examples, 106823 words/s, in_qsize 20, out_qsize 0
2021-03-28 11:14:00,461 : INFO : worker thread finished; awaiting finish of 11 more threads
2021-03-28 11:14:00,509 : INFO : worker thread finished; awaiting finish of 10 more threads
2021-03-28 11:14:00,541 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-28 11:14:00,601 : INFO : worker thread finished; awaiting finish of 8 more threads

In [23]:
X = word_averaging_list(model.wv, df.Description)

2021-03-28 11:15:20,350 : WARNING : cannot compute similarity with no input []
2021-03-28 11:15:20,571 : WARNING : cannot compute similarity with no input []
2021-03-28 11:15:21,091 : WARNING : cannot compute similarity with no input []


#### Обучение ближайших соседей

In [24]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_jobs=12)
nn.fit(X)

NearestNeighbors(n_jobs=12)

## Предсказания для мурманских мероприятий

In [25]:
df1 = pd.read_csv('data.csv')

In [26]:
df1.description = df1.description.apply(html2text.html2text)
df1.description = df1.description.str.lower()
df1.description = df1.description.str.replace('[\W_]', ' ', regex=True)
df1.description = df1.description.apply(lambda x: [i for i in x.split() if len(i) > 1])

In [27]:
X_wv = word_averaging_list(model.wv, df1.description)

### Первый пример

#### Мероприятие

In [28]:
n_sample = 0
print(df1.loc[n_sample].description)

['экспозиции', 'выставки', 'будут', 'представлены', 'пейзажи', 'города', 'исполнении', 'мурманских', 'художников', 'из', 'частной', 'коллекции', 'алексея', 'макарова', 'алексей', 'макаров', '1997', 'по', 'настоящее', 'время', 'ведущий', 'актер', 'драматического', 'театра', 'северного', 'флота', 'на', 'выставке', 'зрители', 'увидят', 'портреты', 'мурманска', 'исполнении', 'разных', 'художников', 'виктории', 'зубицкой', 'татьяны', 'ковалёвой', 'николая', 'ковалёва', 'николая', 'морозова', 'виталия', 'бубенцова', 'анатолия', 'шаковца', 'анастасии', 'берёзы', 'никиты', 'ковалёва', 'других', 'известных', 'художников', 'мурманска', 'мурманской', 'области', 'источник', 'культура', 'рф', 'https', 'culture', 'ru']


#### Похожие на данное мероприятие

In [37]:
dist, nums = nn.kneighbors(X_wv[None, n_sample], 5)
print(*df.iloc[nums[0]].Description.str[:139], sep='\n_______\n')

['знаменательному', 'празднику', 'победы', 'великой', 'отечественной', 'войне', 'организаторы', 'московского', 'международного', 'фестиваля', 'круг', 'света', 'приготовили', 'особое', 'шоу', 'образы', 'военных', 'лет', 'оживут', 'на', 'фасадах', 'центрального', 'академического', 'театра', 'российской', 'армии', 'белорусского', 'вокзала', 'представления', 'на', 'патриотическую', 'тему', 'можно', 'будет', 'увидеть', 'каждый', 'вечер', 'по', '10', 'мая', 'бесплатно', 'для', 'их', 'создания', 'использовались', 'редкие', 'кадры', 'из', 'старинной', 'кинохроники', 'новейшие', 'технологии', 'все', 'это', 'должно', 'произвести', 'мощный', 'эмоциональный', 'эффект', 'подарить', 'зрителям', 'большое', 'визуальное', 'приключение', 'театр', 'российской', 'армии', '10', 'мая', 'на', 'фасаде', 'здания', 'театра', 'http', 'kudago', 'com', 'msk', 'place', 'teatr', 'rossijskoj', 'armii', '21', '30', 'до', '23', '00', 'течение', 'трех', 'дней', 'будут', 'транслироваться', 'два', 'световых', 'представлен

### Второй пример

#### Мероприятие

In [31]:
n_sample = 15
print(df1.loc[n_sample].description[:429])

['ежегодно', 'середине', 'лета', 'на', 'берегу', 'океана', 'принимает', 'гостей', 'арктический', 'фестиваль', 'териберка', 'известные', 'российские', 'музыканты', 'живые', 'выступления', 'незабываемая', 'атмосфера', 'праздника', 'притягивают', 'сюда', 'тысячи', 'туристов', 'фишка', 'фестиваля', 'открытые', 'мастер', 'классы', 'именитых', 'шеф', 'поваров', 'лучшие', 'из', 'лучших', 'здесь', 'соревнуются', 'мастерстве', 'угощают', 'гостей', 'блюдами', 'арктической', 'кухни', 'морские', 'деликатесы', 'свежайшие', 'морепродукты', 'северные', 'ягоды', 'необычное', 'мясо', 'ресторанной', 'подачей', 'настоящий', 'рай', 'для', 'гурманов', '2019', 'году', 'фестиваль', 'посетили', 'около', 'пяти', 'тысяч', 'человек']


#### Похожие на данное мероприятие

In [32]:
dist, nums = nn.kneighbors(X_wv[None, n_sample], 5)
print(*df.iloc[nums[0]].Description.str[:250], sep='\n_______\n')

['фестиваль', 'красок', 'холи', 'проводится', 'киеве', 'уже', 'четвёртый', 'раз', 'собирает', 'год', 'от', 'года', 'всё', 'больше', 'участников', 'расширяется', 'программа', 'праздника', 'теперь', 'холи', 'это', 'не', 'просто', 'яркие', 'краски', 'фестиваль', 'https', 'kudago', 'com', 'kev', 'festivals', 'превратился', 'настоящий', 'гимн', 'дружбе', 'лету', 'этом', 'году', 'пришедшие', 'на', 'ледовый', 'стадион', 'поучаствуют', 'большой', 'битве', 'красками', 'зарядятся', 'позитивными', 'эмоциями', 'во', 'время', 'увлекательных', 'флешмобов', 'зажгут', 'танцпол', 'под', 'ритмы', 'от', 'самых', 'классных', 'диджеев', 'не', 'забывайте', 'про', 'традиционные', 'обнимашки', 'free', 'hugs', 'программе', 'четвёртого', 'фестиваля', 'красок', 'холи', 'также', 'много', 'полезных', 'развлечений', 'на', 'стадионе', 'откроется', 'ярмарка', 'ремесленников', 'пройдут', 'мастер', 'классы', 'от', 'профи', 'конкурсы', 'предложат', 'свои', 'слуги', 'мастера', 'боди', 'арта', 'мехенди', 'для', 'самых', '

### Третий пример

#### Мероприятие

In [33]:
n_sample = 25
print(df1.loc[n_sample].description[:402])

['одна', 'из', 'самых', 'героических', 'романтических', 'морских', 'профессий', 'моряк', 'подводник', 'об', 'отличающей', 'их', 'отваге', 'полной', 'самоотдаче', 'делу', 'железной', 'дисциплине', 'морском', 'товариществе', 'сложено', 'немало', 'песен', 'которыми', 'библиотекари', 'предлагают', 'познакомиться', 'ходе', 'встречи', 'прозвучит', 'стихотворение', 'фисановича', 'строевая', 'подводная', 'ставшее', 'неофициальной', 'песней', 'гимном', 'подводников', 'годы', 'великой', 'отечественной', 'войны', 'присутствующим', 'будет', 'рассказано', 'об', 'истории', 'создания', 'известной', 'музыкальной', 'композиции', 'усталая', 'подлодка', 'пахмутовой', 'добронравова', 'источник', 'культура', 'рф', 'https', 'culture', 'ru']


#### Похожие на данное мероприятие

In [34]:
dist, nums = nn.kneighbors(X_wv[None, n_sample], 5)
print(*df.iloc[nums[0]].Description.str[:250], sep='\n_______\n')

['знаменательному', 'празднику', 'победы', 'великой', 'отечественной', 'войне', 'организаторы', 'московского', 'международного', 'фестиваля', 'круг', 'света', 'приготовили', 'особое', 'шоу', 'образы', 'военных', 'лет', 'оживут', 'на', 'фасадах', 'центрального', 'академического', 'театра', 'российской', 'армии', 'белорусского', 'вокзала', 'представления', 'на', 'патриотическую', 'тему', 'можно', 'будет', 'увидеть', 'каждый', 'вечер', 'по', '10', 'мая', 'бесплатно', 'для', 'их', 'создания', 'использовались', 'редкие', 'кадры', 'из', 'старинной', 'кинохроники', 'новейшие', 'технологии', 'все', 'это', 'должно', 'произвести', 'мощный', 'эмоциональный', 'эффект', 'подарить', 'зрителям', 'большое', 'визуальное', 'приключение', 'театр', 'российской', 'армии', '10', 'мая', 'на', 'фасаде', 'здания', 'театра', 'http', 'kudago', 'com', 'msk', 'place', 'teatr', 'rossijskoj', 'armii', '21', '30', 'до', '23', '00', 'течение', 'трех', 'дней', 'будут', 'транслироваться', 'два', 'световых', 'представлен